# L'Analyse Sémantique Latente
Dans ce projet, nous allons voir comment « extraire » des concepts à partir d'un corpus (collection) de documents textuels.

Je vais vous montrer comment extraire mathématiquement des "concepts" de ce corpus. La technique que nous allons utiliser s'appelle "l'analyse sémantique latente".

In [539]:
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import requests as req

J'ai scrapé un article sur le **Deep Learning** à travers le site **CNRS Le Journal**<br>
[https://lejournal.cnrs.fr/billets/letonnante-acceptabilite-des-deep-fake](https://lejournal.cnrs.fr/billets/letonnante-acceptabilite-des-deep-fake)

In [540]:
# url_article = 'https://www.ledevoir.com/culture/medias/601874/medias-l-esprit-de-i-science-i-amp-i-vie-i-reinvente-dans-le-nouveau-magazine-i-epsiloon-i'
url_article = 'https://lejournal.cnrs.fr/billets/letonnante-acceptabilite-des-deep-fake'

In [541]:
reponse = req.get(url_article)

In [542]:
article = ''
if reponse.ok:
    soup = BeautifulSoup(reponse.text)
    
    articleText = soup.findAll('p')
    
    # divArt = soup.findAll('div')
    
    articleDocs = [paragraphe.text for paragraphe in articleText if len(paragraphe)>4]
    article = [paragraphe.lower() for paragraphe in articleDocs]
    
    # for i in divArt:
    #     if len(i)>5:
    #         article.append(i.text.lower())
else:
    print('Pas de réponse')

In [543]:
import htmlentities as decodeEnt

In [544]:
def supprimer_nomber(texte):
    import re
    for i in re.findall('\d{1,}', texte):
        texte = texte.replace(i, '')
    return texte

In [545]:
def preproces(docs, entitie, separateur):
    docs = [supprimer_nomber(art) for art in docs ]
    return [art.replace(entitie, separateur) for art in docs ]

In [546]:
article = preproces(article, '\xa0', ' ')
article = preproces(article, '\t', ' ')
article = preproces(article, '\n', ' ')

Les mots vides sont des mots que je ne veux pas convertir en fonctionnalités, car ils ne sont pas particulièrement utiles. Des mots comme **a**, **and** et **the** sont de bons mots vides en français. Je peux utiliser une liste intégrée de mots vides de nltk pour commencer. Ensuite, j'ajouterai des mots vides personnalisés qui sont **html indésirable** que je dois nettoyer de mes données

In [547]:
mot_inutils = set(stopwords.words('french'))

In [548]:
len(mot_inutils)

157

## TF-IDF Vectorizing
Je vais utiliser le vectoriseur TF-IDF de scikit-learn pour prendre mon corpus et convertir chaque document en une matrice creuse de fonctionnalités TFIDF...

In [549]:
article[0]

'une fois par mois, retrouvez sur notre site les inédits du cnrs, des analyses scientifiques originales publiées en partenariat avec libération.  '

In [550]:
vectorizer = TfidfVectorizer(
                             stop_words=mot_inutils, use_idf=True, 
                             ngram_range=(1, 5)
)

In [551]:
X = vectorizer.fit_transform(article)

In [552]:
X[0]

<1x2518 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [553]:
# print(X[1])

## LSA
* **Entrée : X**, une matrice où m est le nombre de documents que j'ai, et n est le nombre de termes

* **Processus :** Je vais décomposer X en matrices appelées U, S et T. Lorsque nous effectuons la décomposition, nous devons choisir une valeur k, c'est le nombre de concepts que nous allons conserver.
<div style="width:150px; margin: 10px auto;">X = ${USV}^T$</div>
* **U :** sera une matrice mxk. Les lignes seront des documents et les colonnes seront des **concepts**
* **S :** sera une matrice kxk. Les éléments seront la quantité de variation capturée à partir de chaque **concept**
* **V :** sera une matrice mxk (attention à la transposition). la ligne sera les termes et les colonnes seront **concepts**


In [554]:
X.shape[0], X.shape[1]

(7, 2518)

In [555]:
lsa = TruncatedSVD(n_components=X.shape[0], n_iter=100)

In [556]:
model = lsa.fit(X)

In [557]:
V = model.components_[0]
V

array([0.01748629, 0.01748629, 0.01748629, ..., 0.01920119, 0.01920119,
       0.01920119])

In [558]:
termes = vectorizer.get_feature_names()

for i, comp in enumerate(model.components_):
    
    termeEtConc = zip(termes, comp)
    
    sortedTermes = sorted(termeEtConc, key=lambda x: x[1], reverse=True)[:3]
    print(f'Concept {i}')
    for terme in sortedTermes:
        print(f' - {terme[0]}')
        print(' ')
    

Concept 0
 - voix
 
 - plus
 
 - comme
 
Concept 1
 - site
 
 - algorithmes contre
 
 - algorithmes contre images
 
Concept 2
 - technologies
 
 - transformations vocales
 
 - vocales
 
Concept 3
 - deep
 
 - deep fake
 
 - fake
 
Concept 4
 - analyses
 
 - analyses scientifiques
 
 - analyses scientifiques originales
 
Concept 5
 - montré
 
 - participants
 
 - tout
 
Concept 6
 - scénarios
 
 - voix
 
 - situations
 


In [559]:
"""
  Technologie, 
  les  algorithmes contre images,
  les application des transformations vocales,
  analyses scientifiques
"""

'\n  Technologie, \n  les  algorithmes contre images,\n  les application des transformations vocales,\n  analyses scientifiques\n'

In [560]:
t = "145 852 les application des transformations vocale"

In [535]:
supprimer_nomber(t)

'  les application des transformations vocale'